In [1]:
!pip install transformers sentence_transformers pandas nltk numpy

In [2]:
import pandas as pd

# 读取三个 CSV 文件
train_df = pd.read_csv('cross_domains_cross_models/train.csv')
test_df = pd.read_csv('cross_domains_cross_models/test.csv')
valid_df = pd.read_csv('cross_domains_cross_models/valid.csv')

# 按行合并三个 DataFrame
combined_df = pd.concat([train_df, test_df, valid_df], ignore_index=True)

# 保存为新的 CSV 文件
combined_df.to_csv('data/data.csv', index=False)


In [11]:
import hashlib 
import pandas as pd
import os

deepfake_name_dct = {'OpenAI-GPT':['gpt-3.5-trubo','text-davinci-002', 'text-davinci-003'],
            'Meta-LLaMA':['13B', '30B', '65B', '7B'],
            'GLM-130B':['GLM130B'],
            'Google-FLAN-T5':['flan_t5_base', 'flan_t5_large','flan_t5_small', 'flan_t5_xl', 'flan_t5_xxl'],
            'Facebook-OPT':['opt_1.3b', 'opt_125m', 'opt_13b', 'opt_2.7b', 'opt_30b', 'opt_350m', 'opt_6.7b', 'opt_iml_30b','opt_iml_max_1.3b'],
            'BigScience':['bloom_7b','t0_11b', 't0_3b'],
            'EleutherAI':['gpt_j','gpt_neox'],
            'human':['human']}
deepfake_model_set ={'OpenAI-GPT':0,'Meta-LLaMA':1,'GLM-130B':2,'Google-FLAN-T5':3,
            'Facebook-OPT':4,'BigScience':5,'EleutherAI':6,'human':7}

def stable_long_hash(input_string):
    hash_object = hashlib.sha256(input_string.encode())
    hex_digest = hash_object.hexdigest()
    int_hash = int(hex_digest, 16)
    long_long_hash = (int_hash & ((1 << 63) - 1))
    return long_long_hash

def process_data(dataset):
    data_list=[]
    for i in range(len(dataset)):
        text,label,src=dataset[i]['text'],str(dataset[i]['label']),dataset[i]['src']
        data_list.append((text,label,src,stable_long_hash(text)))
    return data_list

def load_deepfake(data_file='data.csv'):
    """
    读取合并后的 data.csv 并返回处理后的列表
    """
    if not os.path.exists(data_file):
        raise FileNotFoundError(f"{data_file} 不存在，请先生成 data.csv")
    
    df = pd.read_csv(data_file, usecols=["text", "label", "src"])
    
    # 转换为字典列表
    data_dict_list = []
    for i in range(len(df)):
        dct = {
            'text': df.loc[i, 'text'],
            'label': df.loc[i, 'label'],
            'src': df.loc[i, 'src']
        }
        data_dict_list.append(dct)
    
    processed_data = process_data(data_dict_list)
    
    return processed_data


In [12]:
# 读取数据
data = load_deepfake('data/data.csv')

# 转成 DataFrame 并显示前10条
df = pd.DataFrame(data)
df.head(10)

,0,1,2,3
0,Brian worked hard all summer during his break....,0,roct_machine_continuation_opt_1.3b,886295285662808058
1,"The 18-year-old can play at right-back, centre...",0,xsum_machine_continuation_opt_iml_max_1.3b,4766430349215878977
2,then steps back The man then steps back and wa...,0,hswag_machine_topical_text-davinci-002,2690681410079157316
3,I woke up on that burning summer morning as if...,1,wp_human,6893513638684945537
4,Capt. Gale waited patiently at the entrance to...,0,wp_machine_continuation_t0_11b,3613363184767679408
5,We got here before opening (they open an hour ...,0,yelp_machine_continuation_flan_t5_small,9216918736741562764
6,If you are ready to move on or don't like the ...,1,hswag_human,4429398717319019231
7,Concerns were raised over whether Levi's Stadi...,0,squad_machine_continuation_bloom_7b,6038944168635117040
8,Different people may have different background...,0,hswag_machine_continuation_7B,1394438369172459634
9,We study the bandit problem where arms are ass...,0,sci_gen_machine_continuation_30B,8028078670725533133
